In [182]:
from typing import Tuple, NamedTuple, Any, List
import textwrap
import functools

import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.feature_selection import RFE
import shap
import seaborn as sns
import matplotlib.pyplot as plt

In [111]:
# We offer pre-filtered Google Drive files to speed up execution of this cell.

#df = pd.read_csv("user_log_format1.csv")  # SHA256: a6bc8eabc40b61a562a5e45c91e9b822418d0d863576b28a9ccd549451e55a92
#df = df[df['item_id'].isin(range(481, 640+1))]
df_user_log = pd.read_csv('https://drive.google.com/uc?id=1DwEvKOYl0hQmd2muwDa13O17smPPgn0D')
df_user_info = pd.read_csv('https://drive.google.com/uc?id=1wM9LY4rAsMLGe6GUZNQd50R8WHiH9xaf')
#df1 = pd.read_csv("user_info_format1.csv")
df2 = pd.read_csv("train_format1.csv")

In [197]:
merged_df = pd.merge(pd.merge(df_user_log, df_user_info, on='user_id'), df2, on='user_id')
merged_df = merged_df[merged_df.notnull().all(axis=1)]  # Remove rows with any nulls

merged_df['time_stamp'].astype('str')
merged_df['time_str'] = merged_df['time_stamp'].astype('str')
merged_df['day'] = merged_df['time_stamp'] % 100
merged_df['month'] = (merged_df['time_stamp'] - merged_df['day']) / 100
merged_df['month'] = merged_df['month'].astype('int')
merged_df['day_of_year'] = merged_df['month'].apply(lambda x: sum([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31][:x - 1])) + merged_df['day']  # If we have to deal with leap years, I will cry
merged_df['day_of_week'] = merged_df['day'] % 7
merged_df

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,merchant_id,label,time_str,day,month,day_of_year,day_of_week
0,69247,568,683,270,397.0,1110,0,4.0,1.0,1999,0,1110,10,11,314,3
1,69247,568,683,270,397.0,1110,0,4.0,1.0,2954,0,1110,10,11,314,3
2,327584,489,420,3845,3156.0,708,0,4.0,0.0,2397,0,708,8,7,189,1
3,91170,640,612,2246,394.0,1111,0,5.0,0.0,1742,0,1111,11,11,315,4
4,290029,495,464,4535,1720.0,1008,2,4.0,1.0,4585,0,1008,8,10,281,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089,280765,496,267,2115,6973.0,1101,0,0.0,0.0,4765,0,1101,1,11,305,1
5090,280765,496,267,2115,6973.0,1101,0,0.0,0.0,4765,0,1101,1,11,305,1
5091,87449,504,1213,2264,3275.0,1015,0,0.0,1.0,1816,0,1015,15,10,288,1
5092,87449,504,1213,2264,3275.0,1015,0,0.0,1.0,4518,0,1015,15,10,288,1


In [214]:
def merge_multiple(*dfs, on='user_id', default=0):
    return functools.reduce(lambda x,y: pd.merge(x, y, how='outer').fillna(default, downcast='infer'), dfs)

In [215]:
# How many times has the user interacted with us, in general?
user_interaction_count = merged_df.value_counts(subset='user_id').rename("user_interaction_count").reset_index()

# How many unique times (invariant of action_type) has the user interacted with us?
# Also can be interpreted as How many unique separate days (without regrad to action_type) has the user interacted with us?
user_uniq_interaction_count = merged_df.drop(columns='action_type').drop_duplicates().value_counts(subset='user_id').rename("user_uniq_interaction_count").reset_index()
merge_multiple(user_interaction_count, user_uniq_interaction_count)

,user_id,user_interaction_count,user_uniq_interaction_count
0,135731,120,36
1,174817,40,8
2,218756,40,4
3,321987,28,1
4,183753,24,4
...,...,...,...
2097,195154,1,1
2098,195114,1,1
2099,194291,1,1
2100,194212,1,1


In [216]:
# How many times has the user interacted with us on that month?
user_month_count = merged_df.value_counts(subset=['user_id', 'month']).rename("user_month_count").reset_index()

# How many times has the user bought something from us on that month?
user_month_purchase_count = merged_df[merged_df['action_type'] == 2].value_counts(subset=['user_id', 'month']).rename("user_month_purchase_count").reset_index()

# How many times has the user interacted with us on that day?
user_day_count = merged_df.value_counts(subset=['user_id', 'day_of_year']).rename("user_day_count").reset_index()
merge_multiple(user_month_count, user_month_purchase_count, user_day_count)

,user_id,month,user_month_count,user_month_purchase_count,day_of_year,user_day_count
0,135731,9,120,12,268,72
1,135731,9,120,12,265,36
2,135731,9,120,12,269,12
3,174817,6,40,8,176,36
4,174817,6,40,8,167,4
...,...,...,...,...,...,...
2640,188649,11,1,0,310,1
2641,188321,11,1,0,315,1
2642,187072,11,1,0,309,1
2643,186856,11,1,0,313,1


In [236]:
# How many times has the user interacted with the merchant?
user_merchant_interaction_count = merged_df.value_counts(subset=['user_id', 'merchant_id']).rename("user_merchant_interaction_count").reset_index()

# How many unique times has the user interacted with the merchant?
user_uniq_merchant_interaction_count = merged_df.drop(columns='action_type').drop_duplicates().value_counts(subset=['user_id', 'merchant_id']) \
        .rename("user_uniq_merchant_interaction_count").reset_index()

# How many times has the user bought something from the merchant?
user_merchant_purchase_count = merged_df[merged_df['action_type'] == 2].value_counts(subset=['user_id', 'merchant_id']) \
        .rename("user_merchant_purchase_count").reset_index()

# How many times has the user favorited something the merchant sells?
user_merchant_favorite_count = merged_df[merged_df['action_type'] == 3].value_counts(subset=['user_id', 'merchant_id']) \
        .rename("user_merchant_favorite_count").reset_index()

merge_multiple(user_merchant_interaction_count, user_uniq_merchant_interaction_count, user_merchant_purchase_count, user_merchant_favorite_count)

,user_id,merchant_id,user_merchant_interaction_count,user_uniq_merchant_interaction_count,user_merchant_purchase_count,user_merchant_favorite_count
0,321987,1810,28,1,0,0
1,394258,200,24,5,4,0
2,376488,4607,18,6,2,0
3,132468,747,13,3,1,1
4,340967,3609,13,2,3,0
...,...,...,...,...,...,...
2913,171268,2636,1,1,0,0
2914,171556,1768,1,1,0,0
2915,171654,73,1,1,0,1
2916,171998,4818,1,1,0,0


In [238]:
# How many times has the user interacted with the item?
user_item_interaction_count = merged_df.value_counts(subset=['user_id', 'item_id']).rename("user_item_interaction_count").reset_index()

# How many unique times has the user interacted with the item?
user_uniq_item_interaction_count = merged_df.drop(columns='action_type').drop_duplicates().value_counts(subset=['user_id', 'item_id']) \
        .rename("user_uniq_item_interaction_count").reset_index()

# How many times has the user bought the item?
user_item_purchase_count = merged_df[merged_df['action_type'] == 2].value_counts(subset=['user_id', 'item_id']) \
        .rename("user_item_purchase_count").reset_index()

# How many times has the user favorited the item?
user_item_favorite_count = merged_df[merged_df['action_type'] == 3].value_counts(subset=['user_id', 'item_id']) \
        .rename("user_item_favorite_count").reset_index()

merge_multiple(user_item_interaction_count, user_uniq_item_interaction_count, user_item_purchase_count, user_item_favorite_count)

,user_id,item_id,user_item_interaction_count,user_uniq_item_interaction_count,user_item_purchase_count,user_item_favorite_count
0,135731,592,120,36,12,12
1,218756,611,40,4,4,0
2,174817,517,40,8,8,4
3,321987,628,28,1,0,0
4,394258,517,24,5,4,0
...,...,...,...,...,...,...
2129,188969,483,1,1,0,0
2130,188649,484,1,1,0,0
2131,188321,591,1,1,0,0
2132,187072,617,1,1,0,0


In [231]:
# How many times has the user interacted with the category?
user_cat_interaction_count = merged_df.value_counts(subset=['user_id', 'cat_id']).rename("user_cat_interaction_count").reset_index()

# How many unique times has the user interacted with the category?
user_uniq_cat_interaction_count = merged_df.drop(columns='action_type').drop_duplicates().value_counts(subset=['user_id', 'cat_id']) \
        .rename("user_uniq_cat_interaction_count").reset_index()
merge_multiple(user_cat_interaction_count, user_uniq_cat_interaction_count)

,user_id,cat_id,user_cat_interaction_count,user_uniq_cat_interaction_count
0,135731,1505,120,36
1,218756,177,40,4
2,174817,1255,40,8
3,321987,737,28,1
4,394258,1255,24,5
...,...,...,...,...
2129,188969,351,1,1
2130,188649,883,1,1
2131,188321,369,1,1
2132,187072,748,1,1


In [253]:
# What is the ranking of the merchant in terms of total number of items sold?
# (higher is better)
merchant_sold_rank = merged_df[merged_df['action_type'] == 2].value_counts(subset='merchant_id').rank(method='dense') \
        .rename("merchant_sold_rank").reset_index()

# What is the ranking of the merchant in terms of total number of favorites?
# (higher is better)
merchant_favorite_rank = merged_df[merged_df['action_type'] == 3].value_counts(subset='merchant_id').rank(method='dense') \
        .rename("merchant_favorite_rank").reset_index()

merge_multiple(merchant_sold_rank, merchant_favorite_rank)

,merchant_id,merchant_sold_rank,merchant_favorite_rank
0,1653,12,3
1,71,11,1
2,2954,10,0
3,3984,9,5
4,3828,8,3
...,...,...,...
318,1725,0,1
319,1755,0,1
320,1890,0,1
321,1761,0,1


In [254]:
# What is the ranking of the item in terms of total sold?
# (higher is better)
item_sold_rank = merged_df[merged_df['action_type'] == 2].value_counts(subset='item_id').rank(method='dense') \
        .rename("item_sold_rank").reset_index()

# What is the ranking of the item in terms of total favorited?
# (higher is better)
item_favorite_rank = merged_df[merged_df['action_type'] == 3].value_counts(subset='item_id').rank(method='dense') \
        .rename("item_favorite_rank").reset_index()

merge_multiple(item_sold_rank, item_favorite_rank)

,item_id,item_sold_rank,item_favorite_rank
0,517,16,13
1,548,15,2
2,493,14,12
3,568,13,10
4,591,12,5
...,...,...,...
69,532,0,1
70,519,0,1
71,510,0,1
72,491,0,1


In [255]:
merge_multiple(merged_df, user_interaction_count, user_uniq_interaction_count, user_month_count, user_month_purchase_count, \
               user_day_count, user_merchant_interaction_count, user_uniq_merchant_interaction_count, user_merchant_purchase_count, \
               user_merchant_favorite_count, user_item_interaction_count, user_uniq_item_interaction_count, user_item_purchase_count, \
               user_item_favorite_count, user_cat_interaction_count, user_uniq_cat_interaction_count, merchant_sold_rank, \
               merchant_favorite_rank, item_sold_rank, item_favorite_rank)

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,merchant_id,...,user_item_interaction_count,user_uniq_item_interaction_count,user_item_purchase_count,user_item_favorite_count,user_cat_interaction_count,user_uniq_cat_interaction_count,merchant_sold_rank,merchant_favorite_rank,item_sold_rank,item_favorite_rank
0,69247,568,683,270,397,1110,0,4,1,1999,...,2,2,0,0,2,2,0,0,13,10
1,69247,568,683,270,397,1110,0,4,1,2954,...,2,2,0,0,2,2,10,0,13,10
2,183132,568,683,270,397,1104,0,3,0,2954,...,3,3,0,0,3,3,10,0,13,10
3,7423,568,683,270,397,1111,2,0,0,2954,...,4,2,4,0,4,2,10,0,13,10
4,7423,568,683,270,397,1111,2,0,0,2954,...,4,2,4,0,4,2,10,0,13,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5059,186726,554,1280,3229,6972,1111,0,2,1,2036,...,4,1,0,0,4,1,1,0,0,0
5060,186726,554,1280,3229,6972,1111,0,2,1,2036,...,4,1,0,0,4,1,1,0,0,0
5061,186726,554,1280,3229,6972,1111,0,2,1,2036,...,4,1,0,0,4,1,1,0,0,0
5062,186726,554,1280,3229,6972,1111,0,2,1,2036,...,4,1,0,0,4,1,1,0,0,0
